In [1]:
import pandas as pd

df = pd.read_csv("../data/wrime.tsv", sep="\t")
df.columns.tolist()

/Users/nakagawatsubasa/.pyenv/versions/sentiment-analysis/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


['Sentence',
 'UserID',
 'Datetime',
 'Train/Dev/Test',
 'Writer_Joy',
 'Writer_Sadness',
 'Writer_Anticipation',
 'Writer_Surprise',
 'Writer_Anger',
 'Writer_Fear',
 'Writer_Disgust',
 'Writer_Trust',
 'Reader1_Joy',
 'Reader1_Sadness',
 'Reader1_Anticipation',
 'Reader1_Surprise',
 'Reader1_Anger',
 'Reader1_Fear',
 'Reader1_Disgust',
 'Reader1_Trust',
 'Reader2_Joy',
 'Reader2_Sadness',
 'Reader2_Anticipation',
 'Reader2_Surprise',
 'Reader2_Anger',
 'Reader2_Fear',
 'Reader2_Disgust',
 'Reader2_Trust',
 'Reader3_Joy',
 'Reader3_Sadness',
 'Reader3_Anticipation',
 'Reader3_Surprise',
 'Reader3_Anger',
 'Reader3_Fear',
 'Reader3_Disgust',
 'Reader3_Trust',
 'Avg. Readers_Joy',
 'Avg. Readers_Sadness',
 'Avg. Readers_Anticipation',
 'Avg. Readers_Surprise',
 'Avg. Readers_Anger',
 'Avg. Readers_Fear',
 'Avg. Readers_Disgust',
 'Avg. Readers_Trust']

In [2]:
df_writer = df.loc[:, "Writer_Joy":"Writer_Trust"]
df_reader1 = df.loc[:, "Reader1_Joy":"Reader1_Trust"]
df_reader2 = df.loc[:, "Reader2_Joy":"Reader2_Trust"]
df_reader3 = df.loc[:, "Reader3_Joy":"Reader3_Trust"]
df_avg_readers = df.loc[:, "Avg. Readers_Joy":"Avg. Readers_Trust"]

In [3]:
sentiments = [
    "Joy",
    "Sadness",
    "Anticipation",
    "Surprise",
    "Anger",
    "Fear",
    "Disgust",
    "Trust",
]

In [4]:
df_mean = pd.DataFrame(
    {
        "Writer": df_writer.mean().values,
        "Reader1": df_reader1.mean().values,
        "Reader2": df_reader2.mean().values,
        "Reader3": df_reader3.mean().values,
        "Avg. Readers": df_avg_readers.mean().values,
    },
    index=sentiments,
)
df_mean

,Writer,Reader1,Reader2,Reader3,Avg. Readers
Joy,0.793704,0.399792,0.433380,0.585069,0.464699
Sadness,0.642546,0.237199,0.462917,0.376412,0.351736
Anticipation,0.685764,0.418333,0.404190,0.589236,0.453634
Surprise,0.519375,0.377106,0.217407,0.419398,0.324120
Anger,0.234005,0.047037,0.057315,0.051389,0.049745
Fear,0.297546,0.316412,0.198542,0.341713,0.267639
Disgust,0.376389,0.191806,0.123241,0.411898,0.225972
Trust,0.381782,0.102338,0.050741,0.052685,0.049005


In [5]:
from scipy import stats

for s in sentiments:
    result_ttest = stats.ttest_rel(df[f"Writer_{s}"], df[f"Avg. Readers_{s}"])
    result_anova = stats.f_oneway(
        df[f"Reader1_{s}"],
        df[f"Reader2_{s}"],
        df[f"Reader3_{s}"],
    )

    print(f"[{s}]")
    print(f"Writer-Reader\t{result_ttest}")
    print(f"Reader1-2-3\t{result_anova}")
    print("")

df_writer_all = pd.concat([df[f"Writer_{s}"] for s in sentiments])

df_reader1_all = pd.concat([df[f"Reader1_{s}"] for s in sentiments])

df_reader2_all = pd.concat([df[f"Reader2_{s}"] for s in sentiments])

df_reader3_all = pd.concat([df[f"Reader3_{s}"] for s in sentiments])

df_readers_all = pd.concat([df[f"Avg. Readers_{s}"] for s in sentiments])

result_ttest_all = stats.ttest_rel(df_writer_all, df_readers_all)
result_anova_all = stats.f_oneway(
    df_reader1_all,
    df_reader2_all,
    df_reader3_all,
)

print("[All]")
print(f"Writer-Reader\t{result_ttest_all}")
print(f"Reader1-2-3\t{result_anova_all}")

[Joy]
Writer-Reader	Ttest_relResult(statistic=80.25003417829618, pvalue=0.0)
Reader1-2-3	F_onewayResult(statistic=554.539042468175, pvalue=1.5539080831810054e-240)

[Sadness]
Writer-Reader	Ttest_relResult(statistic=70.79990578608339, pvalue=0.0)
Reader1-2-3	F_onewayResult(statistic=975.2687039310326, pvalue=0.0)

[Anticipation]
Writer-Reader	Ttest_relResult(statistic=51.57500149339286, pvalue=0.0)
Reader1-2-3	F_onewayResult(statistic=643.7468914397273, pvalue=6.3662469253375824e-279)

[Surprise]
Writer-Reader	Ttest_relResult(statistic=47.59330935254756, pvalue=0.0)
Reader1-2-3	F_onewayResult(statistic=920.1743817944918, pvalue=0.0)

[Anger]
Writer-Reader	Ttest_relResult(statistic=63.14743423810323, pvalue=0.0)
Reader1-2-3	F_onewayResult(statistic=10.354422552031124, pvalue=3.187797096974353e-05)

[Fear]
Writer-Reader	Ttest_relResult(statistic=8.805663879488794, pvalue=1.3480061821061374e-18)
Reader1-2-3	F_onewayResult(statistic=563.1797375263448, pvalue=2.95737922915519e-244)

[Disgust